In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install datasets transformers torch evaluate 'transformers[torch]' accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 29.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 44.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 95.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 72.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 74.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 61.6 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 60.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 93.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd



In [4]:
data = pd.read_csv('tinybert_model.csv',index_col=0)

In [5]:
data

# sagemaker => airflow => 모델 학습 및 배포

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [6]:
# 데이터셋의 형태 변환

from datasets import Dataset

dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.3)
dataset


DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', '__index_level_0__'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment', '__index_level_0__'],
        num_rows: 15000
    })
})

In [7]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
label2id = {'negative':0, 'positive':1} # 레이블 문자열 데이터를 숫자로 매핑 딕셔너리

dataset = dataset.map(lambda x: {'label': label2id[x['sentiment']]}) # 레이블 문자열 데이터를 숫자로 매핑

dataset
# def preprocess_function(examples):
#     return tokenizer(examples['text'], truncation=True)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'label'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'label'],
        num_rows: 15000
    })
})

In [9]:
dataset['train'][0]


{'review': "The book is fantastic, this film is not. There is no reason this film could not have embraced a futuristic technological vision of the book. Hell, total recall was released a few years later and that did a good job of it, even a clockwork orange released in the 70s did a good job of trying to make a futuristic world. The bleak German expressionistic colours, the black and white footage from the vision screens, there is no reason for this approach for when the film was made in 1984. The main character is in a white collar writing job yet he dresses like he works with oil and grease in a garage. This film decides to take a mock-communistic approach to set design, atmosphere and theme, yet the novel did not necessarily dictate a communist, worship-the-humble-worker theme itself. This book seriously needs to be adapted in a modern context as this book is more relevant today than ever before. I could not watch more than 20 minutes of this crap. The soundtrack is annoying, the la

In [10]:
%pip install transformers torch # 모델 불러오기 위해 필요

Note: you may need to restart the kernel to use updated packages.


In [11]:
# 데이터 준비가 끝났습니다. => 모델 불러오기

from transformers import AutoTokenizer #GPU
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hugginc Face에서 tinybert 검색 -> huawei-noah 모델 선택
model = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)
tokenizer



config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='huawei-noah/TinyBERT_General_4L_312D', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
tokenizer("Python is easy")

tokenizer(dataset['train'][0]['review'])

{'input_ids': [101, 1996, 2338, 2003, 10392, 1010, 2023, 2143, 2003, 2025, 1012, 2045, 2003, 2053, 3114, 2023, 2143, 2071, 2025, 2031, 14218, 1037, 28971, 10660, 4432, 1997, 1996, 2338, 1012, 3109, 1010, 2561, 9131, 2001, 2207, 1037, 2261, 2086, 2101, 1998, 2008, 2106, 1037, 2204, 3105, 1997, 2009, 1010, 2130, 1037, 5119, 6198, 4589, 2207, 1999, 1996, 17549, 2106, 1037, 2204, 3105, 1997, 2667, 2000, 2191, 1037, 28971, 2088, 1012, 1996, 21657, 2446, 3670, 6553, 8604, 1010, 1996, 2304, 1998, 2317, 8333, 2013, 1996, 4432, 12117, 1010, 2045, 2003, 2053, 3114, 2005, 2023, 3921, 2005, 2043, 1996, 2143, 2001, 2081, 1999, 3118, 1012, 1996, 2364, 2839, 2003, 1999, 1037, 2317, 9127, 3015, 3105, 2664, 2002, 14464, 2066, 2002, 2573, 2007, 3514, 1998, 21956, 1999, 1037, 7381, 1012, 2023, 2143, 7288, 2000, 2202, 1037, 12934, 1011, 4750, 2594, 3921, 2000, 2275, 2640, 1010, 7224, 1998, 4323, 1010, 2664, 1996, 3117, 2106, 2025, 9352, 4487, 25572, 2618, 1037, 4750, 1010, 7425, 1011, 1996, 1011, 15716, 1

In [13]:
dataset['train'][0]['review']

"The book is fantastic, this film is not. There is no reason this film could not have embraced a futuristic technological vision of the book. Hell, total recall was released a few years later and that did a good job of it, even a clockwork orange released in the 70s did a good job of trying to make a futuristic world. The bleak German expressionistic colours, the black and white footage from the vision screens, there is no reason for this approach for when the film was made in 1984. The main character is in a white collar writing job yet he dresses like he works with oil and grease in a garage. This film decides to take a mock-communistic approach to set design, atmosphere and theme, yet the novel did not necessarily dictate a communist, worship-the-humble-worker theme itself. This book seriously needs to be adapted in a modern context as this book is more relevant today than ever before. I could not watch more than 20 minutes of this crap. The soundtrack is annoying, the lack of fores

In [14]:
def tokenize(batch):  # 토큰화 함수 정의
    # padding=True => 텍스트 데이터를 동일한 길이로 패딩 처리
    # truncation=True => 맥스 토큰 갯수를 넘어가는 문장은 잘라내겠습니다.
    # max_length=300 => 토큰화된 결과의 최대 길이를 300개 (300개 이상의 토큰은 자른다. 300개 이하는 패딩처리)
    temp = tokenizer(batch['review'], padding=True, truncation=True, max_length=300)

    return temp

In [15]:
dataset = dataset.map(tokenize, batched=True, batch_size=None)
dataset

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment', '__index_level_0__', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15000
    })
})

In [16]:
dataset['train'][0]

{'review': "The book is fantastic, this film is not. There is no reason this film could not have embraced a futuristic technological vision of the book. Hell, total recall was released a few years later and that did a good job of it, even a clockwork orange released in the 70s did a good job of trying to make a futuristic world. The bleak German expressionistic colours, the black and white footage from the vision screens, there is no reason for this approach for when the film was made in 1984. The main character is in a white collar writing job yet he dresses like he works with oil and grease in a garage. This film decides to take a mock-communistic approach to set design, atmosphere and theme, yet the novel did not necessarily dictate a communist, worship-the-humble-worker theme itself. This book seriously needs to be adapted in a modern context as this book is more relevant today than ever before. I could not watch more than 20 minutes of this crap. The soundtrack is annoying, the la

In [17]:

from transformers import AutoModelForSequenceClassification  

id2label = {0:'negative', 1:'positive'}

model = AutoModelForSequenceClassification.from_pretrained(
    model, #  사전 학습된 모델의 이름
    num_labels=len(label2id), # 레이블 갯수
    label2id=label2id, # 레이블 문자열 데이터를 숫자로 매핑 딕셔너리
    id2label=id2label
)

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
%pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [19]:
import evaluate
import numpy as np
# Hugging Face evaluate 라이브러리에서 accuracy(정확도) 메트릭을 불러옵니다.
accuracy = evaluate.load('accuracy')
# compute_metrics 함수는 모델의 예측값과 실제 레이블을 받아 정확도를 계산합니다.
def compute_metrics(eval_pred):  #평가 지표 계산 함수 정의
    # eval_pred는 모델의 예측값과 실제 레이블을 포함하는 튜플입니다.
    predictions, labels = eval_pred  # predictions: 모델이 예측한 값, labels: 실제 정답 레이블
    
    # predictions는 모델이 출력한 확률값 배열이며, 가장 높은 확률을 가진 클래스를 선택합니다.
    # np.argmax는 각 샘플에 대해 가장 높은 확률값을 가진 클래스의 인덱스를 반환합니다.
    predictions = np.argmax(predictions, axis=1)  # 각 샘플별로 가장 높은 확률을 가진 클래스 선택
    
    # accuracy.compute는 예측값(predictions)과 실제값(labels)을 비교하여 정확도를 계산합니다.
    # predictions: 모델이 예측한 클래스, references=labels: 실제 정답 클래스
    return accuracy.compute(predictions=predictions, references=labels) #예측값과 실제값 비교 후 정확도 계산

In [20]:
# # !pip install tf-keras
# %pip install 'transformers[torch]'
# %pip install accelerate -U

# # pip freeze

In [21]:
# 학습 및 평가 설정
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir='train_dir', # 학습 결과를 저장할 디렉토리
    overwrite_output_dir=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32, # 각 디바이스당 학습 배치 크기
    per_device_eval_batch_size=32, # 각 디바이스당 평가 배치 크기
    eval_strategy='epoch' # 매 에포크마다 평가
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics, # 평가 지표를 계산하 함수
    tokenizer=tokenizer # 텍스트를 토큰으로 변환하는 도구
)

/tmp/ipykernel_1152836/1138123528.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.354100,0.313499,0.865133
2,0.295900,0.285361,0.878800
3,0.257400,0.290852,0.883200


TrainOutput(global_step=3282, training_loss=0.3195798131186086, metrics={'train_runtime': 255.0332, 'train_samples_per_second': 411.711, 'train_steps_per_second': 12.869, 'total_flos': 882184338000000.0, 'train_loss': 0.3195798131186086, 'epoch': 3.0})

In [23]:
trainer.evaluate()  # 데이터 정확도 확인으로 평가 

{'eval_loss': 0.2908519208431244,
 'eval_accuracy': 0.8832,
 'eval_runtime': 14.2634,
 'eval_samples_per_second': 1051.645,
 'eval_steps_per_second': 32.881,
 'epoch': 3.0}

In [24]:
trainer.save_model('tinybert_sentiment_analysis')  # 옆에 새로운 디렉토리에 모델이 저장될 것임

In [26]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model='tinybert_sentiment_analysis', device=device)
classifier



In [38]:
data = ['i love you', 'come here']
classifier(data)

[{'label': 'positive', 'score': 0.9872484803199768},
 {'label': 'positive', 'score': 0.7877687811851501}]

In [ ]:
vader, nltk, textblob